In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
from webdriver_manager.chrome import ChromeDriverManager
import nltk
from nltk.corpus import stopwords
import string
from tqdm import tqdm
import re

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\remote
[nltk_data]     desktop\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
stop_words = set(stopwords.words('english'))
table = str.maketrans('', '', string.punctuation)
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # Emoticons
                           u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # Transport & map symbols
                           u"\U0001F700-\U0001F77F"  # Alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric shapes
                           u"\U0001F800-\U0001F8FF"  # Supplemental arrows
                           u"\U0001F900-\U0001F9FF"  # Supplemental symbols & pictographs
                           u"\U0001FA00-\U0001FA6F"  # Symbols & pictographs extended-A
                           u"\U0001FA70-\U0001FAFF"  # Symbols & pictographs extended-B
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U000024C2-\U0001F251" 
                           "]+", flags=re.UNICODE)

# Remove stopwords，punctuations，html tags，emojis and transform into lowercase
def preprocess_text(text):
    text = emoji_pattern.sub(r'', text)
    text = text.lower()
    text = text.replace(",", " , ").replace(".", " . ").replace("-", " - ").replace("/", " / ")
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    words = text.split()
    filtered_sentence = " ".join(word.translate(table) for word in words if word not in stop_words)
    return filtered_sentence.strip()

def scrape_imdb_reviews(movie_id):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    url = f"https://www.imdb.com/title/{movie_id}/reviews"
    driver.get(url)

    # Wait for the page to load
    wait = WebDriverWait(driver, 10)

    load_more_clicked = 0
    # Click the btn to load more reviews
    while load_more_clicked < 1:
        try:
            load_more_button = driver.find_element(By.CSS_SELECTOR, "button.ipc-see-more__button")
            # load_more_button.click()
            load_more_button.send_keys('\n')
            wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, ".ipl-load-more__load-indicator")))
            time.sleep(2)
            load_more_clicked +=1
        except Exception as e:
            # print(e)
            break
    print(load_more_clicked)

    # Get the full HTML 
    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()  

    # #Extract movie name
    # movie_name = soup.select_one("[data-testid='subtitle']")

    # Extract review data
    title_reviews = []
    content_reviews = []
    review_date = []

    for review_block in soup.select("article.user-review-item"):
        try:
            title = review_block.select_one("h3.ipc-title__text").text.strip()
            content = review_block.select_one("div.ipc-html-content-inner-div").text.strip()
            date = review_block.select_one("li.review-date").text.strip()
            title_reviews.append(title)
            content_reviews.append(content)
            review_date.append(date)
        except AttributeError:
            print("Skipping a re view that cannot be parsed")

    # check the number of extracted reviews
    print(f"number of review titles: {len(title_reviews)}")
    print(f"number of review content: {len(content_reviews)}")
    print(f"number of review date: {len(review_date)}")

    processed_titles = [preprocess_text(title) for title in title_reviews]
    processed_reviews = [preprocess_text(review) for review in content_reviews]

    reviews_df = pd.DataFrame({
        "Movie ID": movie_id,
        "Review Title": processed_titles,
        "Review Content": processed_reviews,
        "Review Date": review_date
    })

    # csv_filename = f"{movie_id}_Cleaned_Reviews.csv"
    # reviews_df.to_csv(csv_filename, index=False)

    return reviews_df

In [7]:
import re
import pandas as pd
from utils import get_random_movie_id, scrape_worldwide_box_office

scraped = 0
to_scrape = 100
min_votes = 1000  # Only select movies with at least 1000 votes

# Create an empty DataFrame to store all movie reviews
all_reviews_df = pd.DataFrame()

while scraped < to_scrape:
    # Get a random movie ID that meets the minimum votes criteria.
    random_movie_id = get_random_movie_id(min_votes=min_votes)
    
    # Scrape IMDb reviews for the movie.
    reviews_df = scrape_imdb_reviews(random_movie_id)
    
    # If reviews_df is empty, skip this movie and continue.
    if reviews_df.empty:
        continue

    # Try to get the worldwide box office figure; if it fails, skip to the next movie.
    try:
        box_office_str = scrape_worldwide_box_office(random_movie_id)
    except Exception as e:
        continue

    # Extract the number from the box office string, e.g., "$47,680,966" becomes 47680966.
    match = re.search(r'\$([\d,]+)', box_office_str)
    if match:
        box_office_value = int(match.group(1).replace(',', ''))
    else:
        continue  # If no number can be extracted, skip this movie.

    # Add the worldwide box office value as a new column to the reviews DataFrame.
    reviews_df['Worldwide BO'] = box_office_value

    # Append the movie's reviews to the cumulative DataFrame.
    all_reviews_df = pd.concat([all_reviews_df, reviews_df], ignore_index=True)

    scraped += 1
    print(f"Scraped {scraped} movies.")

# Optionally, save the cumulative DataFrame to a CSV file.
all_reviews_df.to_csv("movie_reviews_box_office.csv", index=False)

1
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
number of review titles: 23
number of review content: 23
number of review date: 23


C:\Users\remote desktop\AppData\Local\Temp\ipykernel_16348\434011934.py:22: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


Scraped 1 movies.
0
number of review titles: 0
number of review content: 0
number of review date: 0
0
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
number of review titles: 8
number of review content: 8
number of review date: 8
Scraped 2 movies.
0
Skipping a re view that cannot be parsed
number of review titles: 14
number of review content: 14
number of review date: 14
1
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re vi

C:\Users\remote desktop\AppData\Local\Temp\ipykernel_16348\434011934.py:22: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


Scraped 6 movies.
0
Skipping a re view that cannot be parsed
number of review titles: 23
number of review content: 23
number of review date: 23
0
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
number of review titles: 2
number of review content: 2
number of review date: 2
1
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
number of review titles: 17
number of review content: 17
number of review d

C:\Users\remote desktop\AppData\Local\Temp\ipykernel_16348\434011934.py:22: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


0
Skipping a re view that cannot be parsed
number of review titles: 2
number of review content: 2
number of review date: 2
0
Skipping a re view that cannot be parsed
number of review titles: 6
number of review content: 6
number of review date: 6
0
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
number of review titles: 12
number of review content: 12
number of review date: 12
Scraped 7 movies.
1
Skipping a re view that cannot be parsed
number of review titles: 48
number of review content: 48
number of review date: 48
0
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
number of review titles: 1
number of review content: 1
number of review date: 1
1
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
number of review titles: 46
number of review content: 46
number of review date: 46


C:\Users\remote desktop\AppData\Local\Temp\ipykernel_16348\434011934.py:22: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


Scraped 8 movies.
1
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
number of review titles: 31
number of review content: 31
number of review date: 31
Scraped 9 movies.
0
number of review titles: 11
number of review content: 11
number of review date: 11
Scraped 10 movies.
1
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
number of review titles: 23
number of review content: 23
number of review date: 23


KeyboardInterrupt: 

obs

In [6]:
# Access the IMDB review page
movie_id = "tt6320628" 
scrape_imdb_reviews(movie_id)

5
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re view that cannot be parsed
Skipping a re 

C:\Users\remote desktop\AppData\Local\Temp\ipykernel_47272\3463117793.py:22: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [ ]:
# ✅ Function to get IMDb Top 250 Movie IDs
def get_imdb_top_250():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run in the background
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    url = "https://www.imdb.com/chart/top/"
    driver.get(url)
    time.sleep(3)  # Allow time for the page to load

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    movies = []
    # for movie in soup.select("div.sc-ee514ad1-0.kYZRWL.cli-poster-container"):
    #     movie_title = movie.select_one("h3.ipc-title__text")
    #     movie_id = movie["href"].split("/")[2]  # Extracts the movie ID (e.g., tt0111161)
    #     movies.append({"movie_id": movie_id, "title": movie_title})

    for movie in soup.select("h3.ipc-title__text"):  
        movie_title = movie.text.strip()
        movies.append({"Movie Title": movie_title})

    return movies[:5]  # Return top 5 movies (change this if needed)

top_movies = get_imdb_top_250()

In [14]:
print(len(top_movies))

0


In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

data = []

res = requests.get("https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm.I")
#print(res)
soup = BeautifulSoup(res.content, "html.parser")

for card in soup.select('li.ipc-metadata-list-summary-item'):
    data.append({
        "title": card.select_one('h3.ipc-title__text').text.strip()
        # "year": card.select_one('.titleColumn span').text,
        # 'rating': card.select_one('td[class="ratingColumn imdbRating"]').get_text(strip=True)
            })

df = pd.DataFrame(data)
print(df)
#df.to_csv('out.csv', index=False)

Empty DataFrame
Columns: []
Index: []


In [8]:
# importing the module 
import imdb 
   
# creating instance of IMDb 
ia = imdb.IMDb() 
   
# name  
name = "3 idiots"
   
# searching the name  
search = ia.search_movie(name) 
  
  
# loop for printing the name and id 
for i in range(len(search)): 
      
    # getting the id 
    id = search[i].movieID 
      
    # printing it 
    print(search[i]['title'] + " : " + id ) 

3 Idiots : 1187043
3 Idiots : 3685624
3 Idiots : 28238283
3 Idiots w/GUNS : 0222441
3 Idiots and a Wise Man : 21612358
Three Idiots to a Team : 29720863
Mugguru Monagallu : 15121916
3 Idiots on Wheels : 6689378
Scotch Mist - A Tale of Three English Idiots in Search of Britain's Northernmost Monsters : 31444403
Kidnap in Rome : 1575673
3 Idiots : 12049418
3 Idiots : 33501685
3 Idiot Heroes : 30247415
Three Idiots : 16345748
Three Idiots : 34207697
The Idiots : 0154421
3 Idiots Try Candy! : 8474256
God and 3 Idiots : 25393152
Confessions of Three Idiots : 21124554
Idiots Are People Three! : 2179303


In [ ]:
stop_words = set(stopwords.words('english'))
table = str.maketrans('', '', string.punctuation)
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # Emoticons
                           u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # Transport & map symbols
                           u"\U0001F700-\U0001F77F"  # Alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric shapes
                           u"\U0001F800-\U0001F8FF"  # Supplemental arrows
                           u"\U0001F900-\U0001F9FF"  # Supplemental symbols & pictographs
                           u"\U0001FA00-\U0001FA6F"  # Symbols & pictographs extended-A
                           u"\U0001FA70-\U0001FAFF"  # Symbols & pictographs extended-B
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U000024C2-\U0001F251" 
                           "]+", flags=re.UNICODE)

# Remove stopwords，punctuations，html tags，emojis and transform into lowercase
def preprocess_text(text):
    text = emoji_pattern.sub(r'', text)
    text = text.lower()
    text = text.replace(",", " , ").replace(".", " . ").replace("-", " - ").replace("/", " / ")
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    words = text.split()
    filtered_sentence = " ".join(word.translate(table) for word in words if word not in stop_words)
    return filtered_sentence.strip()

processed_titles = [preprocess_text(title) for title in title_reviews]
processed_reviews = [preprocess_text(review) for review in content_reviews]

reviews_df = pd.DataFrame({
    "Movie Name": movie_name,
    "Review Title": processed_titles,
    "Review Content": processed_reviews,
    "Review Date": review_date
})

csv_filename = f"{movie_id}_Cleaned_Reviews.csv"
reviews_df.to_csv(csv_filename, index=False)

print(reviews_df.head())

NameError: name 'title_reviews' is not defined